<a href="https://colab.research.google.com/github/heeeeeho/BigData_Analyst/blob/main/n414/n414a_Auto_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / Assignment 4*

---
# N414. 신경망과 학습에 관련된 파라미터 튜닝 (HyperTune)



## 실전 연습과제

다음 통신사 고객 이탈(Churn) 데이터셋에서 정확도를 조정해보는 파라미터 학습을 진행해보겠습니다 : [다운로드](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/telecom/TelcomCustomer.csv)

## 진행방식

- 데이터를 다운로드 받고 읽어옴(load)
- 데이터 클리닝을 진행 (필수는 아니지만 추천)
- Keras MLP model을 만들고, 학습 진행
- Hyperparameter 튜닝 진행:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (optimizer에 따라서 해당되면)
 - momentum (optimizer에 따라서 해당되면)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
하이퍼 파라미터의 초기 패스를 위해 그리드 검색 및 교차 검증을 사용할 수 있어야 합니다. 

실제 큰 통신회사의 데이터이기 때문에 최대한 정확하게 파악해 보십시오! 


# 데이터 전처리 

### 문항 1) 내 로컬 파일을 colab에 업로드하기

충분한 GPU를 가지고 있다면, 쉽게 문제를 해결할 수 있겠지만, 제한된 자원에서 충분한 GPU를 제공받지 못할 지 모릅니다. 이럴 때에는 딥러닝 커뮤니티에 물어볼 수도 있겠지만, 가성비 좋은 Colab의 GPU를 이용해서 실제 GPU사용량을 예측할 수 있다면 좋겠습니다. 모델 파라미터의 개수와 batch size 등이 GPU메모리에 큰 영향을 미치니 여러가지로 활용해보시기 바랍니다.

Colab의 GPU를 이용하기 위해서, 로컬로 진행하시던 분들도 이번에는 colab을 사용해봅시다. 

- 구글에서 colab의 라이브러리를 찾아서 업로드하세요. 
이후에 Pandas를 이용하여 데이터프레임으로 저장합니다.

colab을 사용하기위해서는 colab 라이브러리들을 잘 활용할 수 있으면 좋습니다. colab 기본형의 GPU의 사용제한 때문에 하지 못하는 일이 있다면, colab pro를 사용할 수 있습니다. 코드스테이츠에서 제공하는 colab pro 설치 가이드가 있지만, 스스로 문제를 해결해보시면 좋습니다. 그러나 도움이 필요하시면 HelpDesk에 문의해주시기 바랍니다.

### 로컬 파일을 업로드하는 코드를 입력하세요. 

In [ ]:
# 로컬 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving TelcomCustomer.csv to TelcomCustomer.csv


In [ ]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv('TelcomCustomer.csv')

In [ ]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### 문항 2) 결측치가 있는지 isnull()함수를 이용하여 확인하고 결과값을 입력하시오. 

In [ ]:
# 결측치 확인
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### 문항 3) dtypes를 이용해서 데이터 타입을 확인하고 아래 문제에 답하시오.

TotalCharges와 같이 중요한 타켓값이 숫자로 되어있어야 하는데, object로 되어있는 것을 확인하고 숫자형으로 바꿔주세요. <br>
숫자형이 아닌 결측치의 개수는 몇개인가요? 

In [ ]:
# 데이터 타입 확인, TotalCharges 숫자형으로 변환, 결측치 확인, 결측치 드랍
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [ ]:
pd.to_numeric(df['TotalCharges']) # 공백 데이터가 존재하는 것 같다.

ValueError: ignored

In [ ]:
df[df['TotalCharges'].str.contains(' ')] # 공백 데이터 확인

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [ ]:
df.drop(df[df['TotalCharges'].str.contains(' ')].index, inplace=True) # 공백 행 제거

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [ ]:
# customerID 드랍
df = df.drop(columns='customerID')

In [ ]:
# 'No internet service' 로 표현되어 있는 것을 'No'로 변경
no_internet_feats = [ 'TechSupport','OnlineBackup', 'DeviceProtection','StreamingTV',
                 'OnlineSecurity','StreamingMovies']

for i in no_internet_feats:
    df[i] = df[i].replace({'No internet service':'No'})

# 'No phone service' 로 표현되어 있는 것을 'No'로 변경
df['MultipleLines']=df['MultipleLines'].replace({'No phone service':'No'})

# 0으로 표기된 것을 'No'로, 1로 표기된 것을 'Yes'로 변경
df['SeniorCitizen']=df['SeniorCitizen'].replace({0:'No',
                                                 1:'Yes'})

In [ ]:
!pip install category_encoders

In [ ]:
df.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [ ]:
# 원핫 인코딩
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
df_encoded = encoder.fit_transform(df)

In [ ]:
df_encoded.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,29.85,1
1,2,1,2,1,34,2,1,1,2,2,2,1,1,1,2,2,2,56.95,1889.50,1
2,2,1,2,1,2,2,1,1,2,1,1,1,1,1,1,1,2,53.85,108.15,2
3,2,1,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,1840.75,1
4,1,1,2,1,2,2,1,2,1,2,1,1,1,1,1,1,1,70.70,151.65,2


In [ ]:
# 타겟을 0과 1로 바꿔주기
df_encoded['Churn'] = df_encoded['Churn'].replace({1: 0, 2: 1})
df_encoded['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [ ]:
df_encoded.shape

(7032, 20)

### 문항 4) 훈련집합과 테스트 집합을 나누는 코드를 만들고, 해당 코드를 입력하시오.

- random_state=1
- test_size=0.25

In [ ]:
# 훈련, 테스트 셋을 나누기
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_encoded, test_size=0.25, random_state=1)

In [ ]:
# features 와 target 을 분리
target = 'Churn'
features = df_encoded.drop(columns=[target]).columns

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

### 문항 5) sklearn.preprocessing.StandardScaler를 이용하여 정규화를 진행하시고, 문제를 보고 빈칸에 알맞은 단어를 넣으시오.

`X_train_scaled = scaler.##### Your Code Here #####`

In [ ]:
# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled[0]

array([-1.00952584, -0.44660293, -1.03866899, -0.65370807,  1.57743883,
        0.32362629,  1.15393457,  0.17745038, -0.63367288, -1.36841113,
        1.36726855, -0.64691929,  1.25757616,  1.26869981,  1.57728015,
       -0.82893915,  0.60478129,  1.33038512,  2.30352715])

# 모델링

## 기본 모델

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras
import tensorflow as tf
import IPython
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
!pip install -U keras-tuner
import kerastuner as kt

### 문제 6. np.unique를 이용해서 y_train의 Class 의 개수를 확인하고 입력하시오.

In [ ]:
# Class의 개수 확인
y_train.unique()

array([0, 1])

In [ ]:
X_train.shape

(5274, 19)

In [ ]:
# 간단한 모델 만들어서 성능을 보기 !
tf.random.set_seed(7)

model2 = Sequential()
model2.add(Dense(64, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='sigmoid')) # ___분류이므로 노드수 1, 활성화 함수로는 시그모이드(sigmoid)

model2.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model2.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled,y_test))

Epoch 1/10
165/165 [==============================] - 2s 4ms/step - loss: 0.4696 - accuracy: 0.7643 - val_loss: 0.4215 - val_accuracy: 0.7992
Epoch 2/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4240 - accuracy: 0.8032 - val_loss: 0.4191 - val_accuracy: 0.7941
Epoch 3/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4165 - accuracy: 0.8030 - val_loss: 0.4181 - val_accuracy: 0.7964
Epoch 4/10
165/165 [==============================] - 0s 3ms/step - loss: 0.4118 - accuracy: 0.8110 - val_loss: 0.4190 - val_accuracy: 0.7969
Epoch 5/10
165/165 [==============================] - 0s 3ms/step - loss: 0.4075 - accuracy: 0.8093 - val_loss: 0.4201 - val_accuracy: 0.7998
Epoch 6/10
165/165 [==============================] - 0s 3ms/step - loss: 0.4042 - accuracy: 0.8117 - val_loss: 0.4213 - val_accuracy: 0.8003
Epoch 7/10
165/165 [==============================] - 1s 3ms/step - loss: 0.4028 - accuracy: 0.8108 - val_loss: 0.4228 - val_accuracy: 0.7998
Epoch 

In [ ]:
# 테스트셋 사용해서 결과 보기
model2.evaluate(X_test_scaled,  y_test, verbose=2) 

55/55 - 0s - loss: 0.4237 - accuracy: 0.7935 - 82ms/epoch - 1ms/step


[0.42365777492523193, 0.7935153841972351]

파라미터 튜닝을 하기전에 간단히 임의로 넣어본 결과도 꽤 좋습니다. 이젠 GridSearchCV 를 사용해서 튜닝을 해보겠습니다.



## GridSearchCV 사용

In [ ]:
# 모델 만들기
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  model.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

  return model

# keras.wrapper를 활용하여 분류기를 만듭니다
model = KerasClassifier(build_fn=model_builder, verbose=0)

# GridSearch
batch_size = [64, 128, 256]
epochs = [10, 20, 30]
nodes = [64, 128, 256]
activation = ['relu', 'sigmoid']
param_grid = dict(batch_size=batch_size, epochs=epochs, nodes=nodes, activation=activation)


# GridSearch CV를 만들기
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.


In [ ]:
# 최적의 결과값을 낸 파라미터를 출력합니다
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8043230970700582 using {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 20, 'nodes': 256}
Means: 0.7956010699272156, Stdev: 0.0016310698037157122 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'nodes': 64}
Means: 0.7971179485321045, Stdev: 0.0031614320415412757 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'nodes': 128}
Means: 0.7859309713045756, Stdev: 0.006339881198630848 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 10, 'nodes': 256}
Means: 0.7946530183156332, Stdev: 0.006873162711143744 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'nodes': 64}
Means: 0.7895335753758749, Stdev: 0.007210146096207095 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'nodes': 128}
Means: 0.7842244903246561, Stdev: 0.008630866012616485 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 20, 'nodes': 256}
Means: 0.790291984875997, Stdev: 0.009385183960682582 with: {'activation': 'relu', 'batch_size': 64, 'epochs': 30, '

### 문항 7) 최적의 결과를 낸 파라미터와 결과값을 입력해주세요.

- batch_size
- epochs
- nodes
- activation

- 정답은 `[100, 100, 100], activation_name` 형태로 입력해주세요.

다음으로 Keras Tuner 를 사용한 파라미터 튜닝도 해보겠습니다.

In [ ]:
# Best: 0.8043230970700582 using {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 20, 'nodes': 256}

## Keras Tuner 사용

In [ ]:
# 모델 만들기

def model_builder(hp):

  model = Sequential()

  # Dense layer에서 노드 수를 조정(32-512)
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)

  model.add(Dense(units = hp_units, activation='relu'))
  model.add(Dense(units = hp_units, activation='relu'))

  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]합니다. 
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  # 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
  model.compile(optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                loss=keras.losses.BinaryCrossentropy(from_logits = True), 
                metrics=['accuracy'])

  return model

In [ ]:
# 튜너를 인스턴스화하고 하이퍼 튜닝을 수행

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 30, 
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [ ]:
# callback 정의 : 하이퍼 파라미터 검색을 실행하기 전에 모든 교육 단계가 끝날 때마다 교육 출력을 지우도록 콜백을 정의합니다.

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(X_train_scaled, y_train, epochs = 30, batch_size=50, validation_data = (X_test_scaled,y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
최적화된 Dense 노드 수 : {best_hps.get('units')} 
최적화된 Learning Rate : {best_hps.get('learning_rate')} 
""")

Trial 61 Complete [00h 00m 06s]
val_accuracy: 0.8009101152420044

Best val_accuracy So Far: 0.806598424911499
Total elapsed time: 00h 03m 36s

최적화된 Dense 노드 수 : 320 
최적화된 Learning Rate : 0.01 



### 문항 8) Keras 튜너를 활용하여 얻어낸 파라미터를 입력하시오. 

- 정답은 `[100, 100]` 형태로 입력하시오.

In [ ]:
# 최적화된 Dense 노드 수 : 320, 최적화된 Learning Rate : 0.01 

In [ ]:
from tensorflow.keras import regularizers

tf.random.set_seed(1442)
initializer = tf.keras.initializers.HeNormal()

model = Sequential()

model.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer,          
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01))) # L1 norm regularization))
model.add(Dense(best_hps.get('units'),
                activation='relu', kernel_initializer=initializer,            
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

model.compile(optimizer=keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model.fit(X_train_scaled, y_train, epochs=2, batch_size=50, validation_data=(X_test_scaled,y_test))

Epoch 1/2
106/106 [==============================] - 3s 12ms/step - loss: 3.4399 - accuracy: 0.7239 - val_loss: 0.5343 - val_accuracy: 0.7873
Epoch 2/2
106/106 [==============================] - 1s 7ms/step - loss: 0.4993 - accuracy: 0.7945 - val_loss: 0.4903 - val_accuracy: 0.7742


In [ ]:
# 테스트셋 사용해서 결과 보기
model.evaluate(X_test_scaled,  y_test, verbose=2)

55/55 - 0s - loss: 0.4903 - accuracy: 0.7742 - 365ms/epoch - 7ms/step


[0.4902998208999634, 0.774175226688385]